In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.data.spatial.modes import AvirisDataManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.learn.base import LearningModel
import os
from typing import List, Union, Tuple, Optional, Dict, Callable

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"

block_size = 150
method = "aec" # "vae"
model_dims = 32
reprocess_data = False
    
dm.modal.ext =  "_img"
dm.use_model_data = True
dm.proc_type = "skl"
TileManager.block_size = block_size
TileManager.block_index = [0,7]
dm.modal.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]
dm.modal.model_dims = model_dims
dm.modal.reduce_anom_focus = 0.15
dm.modal.reduce_nepoch = 3
dm.modal.reduce_focus_nepoch = 15
dm.modal.reduce_niter = 1
dm.modal.refresh_model = True
dm.modal.modelkey = f"b{block_size}.{method}"
dm.modal.reduce_focus_ratio = 10.0
dm.modal.reduce_dropout = 0.0
dm.modal.reduce_learning_rate = 5e-4
dm.modal.refresh_model = False
dm.modal.modelkey = f"b{block_size}.{method}"

Opening log file:  '/Users/tpmaxwel/.spectraclass/logging/aviris/img_mgr.log'
Using config file: '/Users/tpmaxwel/Development/Projects/spectraclass/defaults/config.py'
Using config file: '/Users/tpmaxwel/.spectraclass/config/aviris/img_mgr.py'


Here we load the project data and define a set of class names (with associated colors) to be used in the classification process.

In [ ]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )
dm.modal.initialize_dimension_reduction( refresh=reprocess_data )

Completed Reading raster file /Users/tpmaxwel/Development/Data/Aviris/ang20170720t004130rfl/ang20170720t004130_rfl_v2p9/ang20170720t004130_corr_v2p9_img, dims = ('band', 'y', 'x'), shape = (425, 11746, 662)
#Tile[0]-> Read Data: shape = (425, 11746, 662), dims=('band', 'y', 'x')
 Preparing inputs
Completed Reading raster file /Users/tpmaxwel/Development/Data/Aviris/ang20170720t004130rfl/ang20170720t004130_rfl_v2p9/ang20170720t004130_corr_v2p9_img, dims = ('band', 'y', 'x'), shape = (425, 11746, 662)
#Tile[0]-> Read Data: shape = (425, 11746, 662), dims=('band', 'y', 'x')
 ---> Writing metadata file at /Volumes/Shared/Cache/spectraclass/aviris/img_mgr/b150.aec.mdata.txt
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 363)]             0         
                                                                 
 dense (Dense)               (None, 182)   

2022-09-13 15:16:11.577276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Completed Reading raster file /Users/tpmaxwel/Development/Data/Aviris/ang20170720t004130rfl/ang20170720t004130_rfl_v2p9/ang20170720t004130_corr_v2p9_img, dims = ('band', 'y', 'x'), shape = (425, 11746, 662)
#Tile[0]-> Read Data: shape = (425, 11746, 662), dims=('band', 'y', 'x')
  ITER[<built-in function iter>]: Processing block(0, 0), data shape = (17657, 363)
Epoch 1/5
69/69 [==============================] - 1s 4ms/step - loss: 0.1890
Epoch 2/5
69/69 [==============================] - 0s 4ms/step - loss: 0.0908
Epoch 3/5
69/69 [==============================] - 0s 4ms/step - loss: 0.0884
Epoch 4/5
69/69 [==============================] - 0s 4ms/step - loss: 0.0869
Epoch 5/5
69/69 [==============================] - 0s 4ms/step - loss: 0.0856
 Trained autoencoder in 2.712965965270996 sec
  ITER[<built-in function iter>]: Processing block(0, 1), data shape = (20491, 363)
Epoch 6/10
81/81 [==============================] - 0s 4ms/step - loss: 0.0580
Epoch 7/10
81/81 [===================

Here we define a custom CNN to use withing the spectraclass framework.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import datasets, layers, models
from spectraclass.learn.models.network import Network, ModelType

class CNN(Network):
    TYPE = ModelType.SPATIAL

    def _build_model(self, **kwargs) -> Tuple[Model,Dict]:
        from spectraclass.learn.models.spatial import SpatialModelWrapper
        nfeatures = kwargs.pop('nfeatures', 32 )
        from spectraclass.model.labels import lm
        input_shape = SpatialModelWrapper.get_input_shape()
        nclasses = lm().nLabels
        ks = kwargs.pop('kernel_size',3)
        model = models.Sequential()
        model.add( Input( shape=input_shape ) )
        model.add( layers.Conv2D( nfeatures, (ks,ks), activation='relu', padding="same" ) )
        model.add( layers.Reshape( SpatialModelWrapper.flatten(input_shape,nfeatures) ) )
        model.add( layers.Dense( nfeatures, activation='relu' ) )
        model.add( layers.Dense( nclasses, activation='softmax' ) )
        return model, kwargs

cm().addNetwork( CNN( 'cnn-1', nfeatures=64, nepochs=200, test_size=0.1 ) )

Here we start up the Spectraclass GUI.

In [ ]:
controller: SpectraclassController = app()
controller.gui()